In [1]:
from aoc2023 import day1
from aoc2023 import maps
import numpy as np
import functools

In [47]:
data = """rn=1,cm-,qp=3,cm=2,qp-,pc=4,ot=9,ab=5,pc-,pc=6,ot=7
"""

In [60]:
data = day1.read_data_string("input.txt")
lines = day1.split_data(data) 
paragraphs = [day1.split_data_p(paragraph) for paragraph in day1.split_data_paragraphs(data)]

In [61]:
data = lines[0].split(",")

In [62]:
def ascii(word): 
    x = 0 
    for letter in word: 
        x += ord(letter)
        x = x * 17
        x = x % 256 
    return x 

In [63]:
ascii("cm")

0

In [64]:
p1 = 0 
for word in data: 
    p1 += ascii(word)

In [65]:
boxes = []
for i in range(256):
    boxes.append([])

In [66]:

for word in data: 
    if '-' in word: 
        b = word.replace('-', '')
        b_number = ascii(b)
        box = boxes[b_number]
        for lens in box: 
            if lens[0] == b: 
                box.remove(lens)

    if '=' in word:
        b, l = word.split('=')
        l_int = int(l)
        b_number = ascii(b)
        box = boxes[b_number]
        found = False
        for lens in box: 
            if lens[0] == b: 
                lens[1] = l_int
                found = True
        if not found:
            box.append([b, l_int])
            

In [67]:
boxes

[[['hll', 8]],
 [['qp', 2]],
 [],
 [['pc', 5], ['vvct', 5]],
 [],
 [['mmk', 1], ['mnqxq', 7]],
 [['nrlrh', 3]],
 [],
 [['zn', 4], ['dlx', 1]],
 [],
 [['qfpqnt', 5]],
 [['hs', 7]],
 [['vv', 9]],
 [['fds', 6], ['tkgg', 4]],
 [['fh', 3], ['tz', 7]],
 [['fcf', 3]],
 [['dl', 2], ['mrq', 3]],
 [['cfx', 6]],
 [['bp', 3], ['pjx', 3]],
 [['pxch', 5], ['bbpmkg', 7], ['chzn', 9]],
 [['hjb', 6], ['pd', 2]],
 [['crp', 3]],
 [['nh', 7], ['zbz', 8], ['jrz', 5], ['xfx', 9]],
 [['fzg', 1]],
 [['ll', 3]],
 [['fxk', 3]],
 [['lzt', 1]],
 [],
 [['fdftx', 3], ['xd', 8], ['ht', 8], ['bnl', 1]],
 [],
 [],
 [['ggfk', 9], ['hlhs', 5]],
 [['bklg', 9]],
 [],
 [['rp', 2]],
 [['qr', 2]],
 [['dftfrn', 3]],
 [],
 [['ccp', 1]],
 [],
 [],
 [['lm', 9]],
 [['fkvc', 8], ['lztp', 4]],
 [['tkl', 1]],
 [['nfx', 6]],
 [],
 [],
 [],
 [],
 [['gfgm', 7]],
 [['nbhz', 2]],
 [['cvj', 4], ['jhq', 2]],
 [],
 [],
 [['nkdjdk', 1]],
 [],
 [],
 [['ml', 8]],
 [],
 [['bbg', 7], ['sxszs', 7]],
 [['jr', 9], ['zb', 7], ['znt', 9]],
 [],
 [['x

In [68]:
p2 = 0 
for b_i in range(len(boxes)): 
    box = boxes[b_i]
    for i in range(len(box)): 
        p2 += box[i][1] * (i+1) *(b_i+1)

In [69]:
p2

221627

In [8]:
len(data)

4000

In [22]:
t_pattern = maps.transpose_matrix(pattern)

In [23]:
def p1(t_pattern): 
    p1 = 0
    for line in t_pattern: 
        force = 0
        max_distance = len(line)
        blocker = 0 
        for i in range(len(line)): 
            x = line[i]
            if x == "O": 
                force += max_distance - blocker
                blocker = blocker + 1 
            if x == '.': 
                pass
            if x == "#": 
                blocker = i + 1 
        p1 += force
    return p1

In [24]:
def move_rocks_left(pattern): 

    p1 = 0
    new_pattern = []
    for line in pattern: 
        force = 0
        max_distance = len(line)
        new_line = ['.'] * max_distance 
        
        blocker = 0 
        for i in range(len(line)): 
            x = line[i]
            if x == "O": 
                force += max_distance - blocker
                new_line[blocker] = "O"
                blocker = blocker + 1 
            if x == '.': 
                pass
            if x == "#": 
                new_line[i] = "#"
                blocker = i + 1 
        p1 += force
        new_pattern.append(new_line)
        
    return new_pattern, p1

In [25]:
def force_now(pattern): 
    
    f = 0
    for line in pattern: 
        force = 0
        max_distance = len(line)
        
        for i in range(len(line)): 
            x = line[i]
            if x == "O": 
                force += max_distance - i
        f += force
    return f
        

In [26]:
def mirror_pattern(pattern): 
    for line in pattern: 
        line.reverse()
    return pattern

In [27]:
def cylce_pattern(p_n): 
    n_p_n, _ = move_rocks_left(p_n)
    p_w = maps.transpose_matrix(n_p_n)
    n_p_w, _ = move_rocks_left(p_w)
    p_s = mirror_pattern(maps.transpose_matrix(n_p_w))
    n_p_s, _ = move_rocks_left(p_s)
    p_e = mirror_pattern(maps.transpose_matrix(n_p_s))
    n_p_e, _ = move_rocks_left(p_e)

    cycled_pattern = mirror_pattern(maps.transpose_matrix(mirror_pattern(n_p_e)))
    force = force_now(cycled_pattern)
    return cycled_pattern, force


In [28]:
from tqdm import tqdm

In [29]:
list_loads = []
for x in tqdm(range(1000)): 
    t_pattern, force = cylce_pattern(t_pattern)
    list_loads.append(force)

100%|██████████| 1000/1000 [00:03<00:00, 298.94it/s]


In [30]:
list_loads

[97447,
 97467,
 97144,
 96947,
 96704,
 96595,
 96476,
 96379,
 96266,
 96210,
 96210,
 96146,
 96061,
 95973,
 95956,
 95923,
 95865,
 95800,
 95708,
 95602,
 95491,
 95398,
 95313,
 95231,
 95144,
 95058,
 94947,
 94864,
 94764,
 94672,
 94545,
 94473,
 94372,
 94247,
 94109,
 93981,
 93883,
 93814,
 93736,
 93652,
 93578,
 93523,
 93460,
 93427,
 93400,
 93378,
 93347,
 93332,
 93331,
 93352,
 93342,
 93305,
 93291,
 93322,
 93291,
 93254,
 93190,
 93179,
 93170,
 93154,
 93129,
 93122,
 93109,
 93093,
 93066,
 93032,
 92994,
 92939,
 92877,
 92830,
 92769,
 92700,
 92629,
 92580,
 92526,
 92467,
 92415,
 92373,
 92317,
 92261,
 92197,
 92148,
 92086,
 92017,
 91966,
 91924,
 91885,
 91831,
 91775,
 91727,
 91648,
 91572,
 91519,
 91473,
 91417,
 91374,
 91335,
 91321,
 91302,
 91292,
 91290,
 91300,
 91328,
 91341,
 91354,
 91343,
 91327,
 91297,
 91277,
 91270,
 91278,
 91295,
 91317,
 91333,
 91332,
 91320,
 91306,
 91286,
 91270,
 91278,
 91295,
 91317,
 91333,
 91332,
 91320,


In [32]:
(1000000000-1000) % 9

0

In [31]:
len([ 91286,
 91270,
 91278,
 91295,
 91317,
 91333,
 91332,
 91320,
 91306,])

9

In [14]:
list_loads

[87,
 69,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
 65,
 64,
 65,
 63,
 68,
 69,
 69,
